In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
!python --V

Unknown option: --
usage: python2 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.


In [2]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

import seaborn as sns
from collections import OrderedDict, Counter
import os
import networkx as nx
import matplotlib.pyplot as plt
from nltk import word_tokenize
import string
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv('/content/drive/My Drive/Projeto Ana/Entrega_2/tweets_total.csv', encoding = 'utf-8')
df = df.dropna()
#df = df[(df['date'] > '2013-05-01 00:00:00') & (df['date'] < '2013-07-31 09:00:00')]

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.



In [0]:
def word_transform(words):
    words = words.split()
    total = [x.lower() for x in words]
    manual_list_stop_words = ['só','disse','ser','sobre','ainda','se','além', 'outros','porque','assim','ter', 'access_time', 'more_horiz',
                             'out','jun','publicado','diz','exame','patrocinado','ago','vip','abr','http','vai','HTTP'
                             'jul','jan','abr','set','nov','dez','conteúdo','mar','maio','estadão','marketing','pictwittercom','dia','mil','pra',
                             'fev','paulo', 'the','após', 'redação', 'foto', 'feira', 'segundo', 'nesta', 'sexta','contra','governo','dilma'
            'onde','pode','sábado','segunda','três', 'fazer','todos','parte','local', 'região', 'disse','quarta','nova','novo',
            'via','veja','neste','nm','aponta','mostra','rt','vejaabrilcombr','folha','vai','desde','do','multimidia','vejacom','blog', '-', '@','"','...','dilma']
    stop_words = stopwords.words('portuguese')
    for i in manual_list_stop_words:
        stop_words.append(i)
    
    total = [x for x in total if x not in stop_words]
    total = [re.sub(r'[,\.!?()]', '', x) for x in total]
    total = [x for x in total if not re.search(r"\d", x)]
    
    long_string = (" ").join(total)
    return long_string

In [5]:
df['tweet'] = df['tweet'].apply(word_transform)
#df['tweet'] = df['tweet'].apply(lambda x: x.split())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: UnicodeWarning:

Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal



In [0]:
df_new = df[['date','tweet']]
df_new.columns = ['Data','Texto']

In [7]:
df_new['Data'] = df_new['Data'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
df_new['Data'] = df_new['Data'].apply(lambda x: x.to_period('M'))
df_new['Data'] = df_new['Data'].apply(lambda x: x.to_timestamp())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

In [0]:
df_new = df_new.groupby(['Data'])['Texto'].apply(' '.join).reset_index()

In [0]:
def contar_palavra_corrupcao(row):
    soma = 0
    for i in row.split():
        if i == 'corrupção':
            soma+=1
    return soma

In [10]:
df_new['coorupcao_freq'] = df_new['Texto'].apply(contar_palavra_corrupcao)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UnicodeWarning:

Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal



In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['coorupcao_freq'], name = 'Corrupção'))
fig.update_xaxes(rangeslider_visible=True)
fig.show()

AttributeError: ignored